# Multi-Perceptron Grid_Search

## Table of Contents

- [Format All Results](#Format-All-Results)


In [2]:
import polars as pl
from icecream import ic
import pandas as pd
from great_tables import GT, md, html, from_column, style, loc
from assignment_3_tools import parquet_to_dict

## Format All Results

In [3]:
result_path = '../../Data/GoogleDrive/MLP_Results/Parameters'

result_pl = {
    key:value.collect().drop('Dataset_Name')
    for key, value in parquet_to_dict(result_path).items()
}

result_pl = {
    key: value if key.endswith('baseline') else value.with_columns(pl.lit("").alias("Grid_Variable"))
    for key, value in result_pl.items()
}

for key, frame in result_pl.items():
    if key.endswith('baseline'):
        continue
    else:
        result_dict = frame.to_dict(as_series=False)
        params = []
        for entry in result_dict['Parameters']:
            param_values = "".join([str(value) for value in entry.values()])
            params.append(param_values)
        result_dict['Parameters'] = params
        result_pl[key] = pl.DataFrame(result_dict)


param_name = dict()
unique_name = list()
for key in result_pl:
    if not key.endswith('baseline'):
        unique_name.append(key[4:])
unique_name = set(unique_name)

for key in unique_name:
        prefix, name = key.split('-', 1)
        param_name.setdefault(name, []).append(key)

result_table = dict()
result_combo = dict()
for param, name_list in param_name.items():
    for name in name_list:
        prefixes = ['test', 'grid']
        for prefix in prefixes:
            alt = f"{prefix}{name}"
            baseline = f"{prefix}_results-baseline"
            alt_df = result_pl[alt].with_columns(pl.col("Parameters").cast(pl.Utf8))
            baseline_dict = result_pl[baseline].to_dict(as_series=False)
            baseline_dict['Parameters'] = [list(val) for test, val in baseline_dict['Parameters'][0].items() if test == param][0]
            baseline_df = pl.DataFrame(baseline_dict).with_columns(pl.col("Parameters").cast(pl.Utf8))
            combo = pl.concat([baseline_df, alt_df])
            combo = combo.with_columns(
                pl.when(pl.col('Grid_Variable') == pl.lit('-baseline'))
                .then(pl.lit("Baseline"))  # New value
                .otherwise(pl.col('Grid_Variable'))
                .alias("Grid_Variable")
            )
            result_combo[alt] = combo
            if prefix == "test":
                table = (
                    GT(combo.sort(by='Recall', descending=True))
                    .tab_header(
                        title=f"{param.title().replace('_', ' ')}",
                        subtitle=f"Test Set Prediction Results"
                    )
                    .tab_spanner(
                        label="Metrics",
                        columns=["Recall", "ROC_AUC", "Accuracy", "Fit_Time"]
                    )
                    .cols_label(
                        Grid_Variable=html(""),
                        Parameters=html(f"{param.title()} <br> Parameter"),
                        ROC_AUC=html("ROC AUC"),
                        Fit_Time=html("Fit Time (s)"),
                    )
                    .fmt_number(columns=["Recall","ROC_AUC","Accuracy"], decimals=2)
                    .fmt_number(columns=["Fit_Time"], decimals=0)
                )
            elif prefix == "grid":
                table = (
                    GT(combo.sort(by='Recall', descending=True))
                    .tab_header(
                        title=f"{param.title().replace('_', ' ')}",
                        subtitle=f"Cross Validation Results"
                    )
                    .fmt_number(columns=["Recall"], decimals=2)
                    .fmt_number(columns=["Fit_Time"], decimals=0)
                    .tab_spanner(
                        label="Metrics",
                        columns=["Recall", "Fit_Time"]
                    )
                    .cols_label(
                        Grid_Variable=html(""),
                        Parameters=html(f"<center>{param.title()} <br> Parameter</center>"),
                        Fit_Time=html("Fit Time (s)")
                    )
                    .cols_align(align="center")
                )
            else:
                continue
            result_table[alt] = table

## Neurons

In [7]:
(
    result_table['test_results_neurons-hidden_layer_sizes']
    .tab_header(title="Neurons of a Single Layer", subtitle="Test Set Prediction Results")
    .cols_label(
        Num_Parameters = html("<center>Total Number of<br>Internal Parameters</center>")
    )
    # .save(file="./tables_and_figures/test_results_neurons-hidden_layer_sizes.png")
)

GT(_tbl_data=shape: (2, 7)
┌───────────────┬────────────┬────────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Num_Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---            ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ i64            ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪════════════════╪══════════╪══════════╪══════════╪════════════╡
│ Baseline      ┆ 100        ┆ 12301          ┆ 0.713213 ┆ 0.808297 ┆ 0.745176 ┆ 193.497407 │
│               ┆ [500]      ┆ 61501          ┆ 0.410034 ┆ 0.719216 ┆ 0.800444 ┆ 1844.94788 │
└───────────────┴────────────┴────────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x2862029d0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Hidden_Layer_Sizes <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Num_Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Total Number of<br>Internal Parameters</center>', type='html'), column_align='right', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Neurons of a Single Layer', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x286203150>, _formats=[<great_tables._gt_data.FormatInfo object at 0x286203290>, <great_tables._gt_data.FormatInfo object at 0x286203690>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='tabl

In [8]:
(
    result_table['grid_results_neurons-hidden_layer_sizes']
    .tab_header(title="Neurons of a Single Layer", subtitle="Cross Validation Results")
    .cols_label(
        Num_Parameters = html("<center>Total Number of<br>Internal Parameters</center>")
    )
    # .save(file="./tables_and_figures/grid_results_neurons-hidden_layer_sizes.png")
)

GT(_tbl_data=shape: (5, 5)
┌───────────────┬────────────┬────────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters ┆ Num_Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---        ┆ ---            ┆ ---      ┆ ---        │
│ str           ┆ str        ┆ i64            ┆ f64      ┆ f64        │
╞═══════════════╪════════════╪════════════════╪══════════╪════════════╡
│               ┆ [500]      ┆ 61501          ┆ 0.955587 ┆ 1844.94788 │
│               ┆ [250]      ┆ 30751          ┆ 0.873436 ┆ 894.794054 │
│ Baseline      ┆ 100        ┆ 12301          ┆ 0.831363 ┆ 193.497407 │
│               ┆ [50]       ┆ 6151           ┆ 0.800942 ┆ 143.82385  │
│               ┆ [1]        ┆ 124            ┆ 0.780501 ┆ 12.700479  │
└───────────────┴────────────┴────────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x286203a90>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='center', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Hidden_Layer_Sizes <br> Parameter</center>', type='html'), column_align='center', column_width=None), ColInfo(var='Num_Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Total Number of<br>Internal Parameters</center>', type='html'), column_align='center', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='center', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Neurons of a Single Layer', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x286203c90>, _formats=[<great_tables._gt_data.FormatInfo object at 0x286203910>, <great_tables._gt_data.FormatInfo object at 0x286208290>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, categor

## Layers

In [9]:
(
    result_table['test_results_layers-hidden_layer_sizes']
    .tab_header(title="Layers With Constant Neurons", subtitle="Test Set Prediction Results")
    .cols_label(
        Num_Parameters = html("<center>Total Number of<br>Internal Parameters</center>")
    )
    # .save(file="./tables_and_figures/test_results_layers-hidden_layer_sizes.png")
)

GT(_tbl_data=shape: (2, 7)
┌───────────────┬──────────────────┬────────────────┬──────────┬──────────┬──────────┬─────────────┐
│ Grid_Variable ┆ Parameters       ┆ Num_Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time    │
│ ---           ┆ ---              ┆ ---            ┆ ---      ┆ ---      ┆ ---      ┆ ---         │
│ str           ┆ str              ┆ i64            ┆ f64      ┆ f64      ┆ f64      ┆ f64         │
╞═══════════════╪══════════════════╪════════════════╪══════════╪══════════╪══════════╪═════════════╡
│ Baseline      ┆ 100              ┆ 12301          ┆ 0.713213 ┆ 0.808297 ┆ 0.745176 ┆ 193.497407  │
│               ┆ [100, 100, 100,  ┆ 42601          ┆ 0.495749 ┆ 0.764613 ┆ 0.812899 ┆ 1696.776056 │
│               ┆ 100]             ┆                ┆          ┆          ┆          ┆             │
└───────────────┴──────────────────┴────────────────┴──────────┴──────────┴──────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x2833e8710>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Hidden_Layer_Sizes <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Num_Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Total Number of<br>Internal Parameters</center>', type='html'), column_align='right', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Layers With Constant Neurons', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x2855dea90>, _formats=[<great_tables._gt_data.FormatInfo object at 0x2855de5d0>, <great_tables._gt_data.FormatInfo object at 0x2861c8090>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='

In [10]:
(
    result_table['grid_results_layers-hidden_layer_sizes']
    .tab_header(title="Layers With Constant Neurons", subtitle="Cross Validation Results")
    .cols_label(
        Num_Parameters = html("<center>Total Number of<br>Internal Parameters</center>")
    )
    # .save(file="./tables_and_figures/grid_results_layers-hidden_layer_sizes.png")
)

GT(_tbl_data=shape: (5, 5)
┌───────────────┬───────────────────────────┬────────────────┬──────────┬─────────────┐
│ Grid_Variable ┆ Parameters                ┆ Num_Parameters ┆ Recall   ┆ Fit_Time    │
│ ---           ┆ ---                       ┆ ---            ┆ ---      ┆ ---         │
│ str           ┆ str                       ┆ i64            ┆ f64      ┆ f64         │
╞═══════════════╪═══════════════════════════╪════════════════╪══════════╪═════════════╡
│               ┆ [100, 100, 100, 100]      ┆ 42601          ┆ 0.980151 ┆ 1696.776056 │
│               ┆ [100, 100, 100, 100, 100] ┆ 52701          ┆ 0.978444 ┆ 1613.07513  │
│               ┆ [100, 100, 100]           ┆ 32501          ┆ 0.963216 ┆ 1606.766589 │
│               ┆ [100, 100]                ┆ 22401          ┆ 0.909198 ┆ 1157.932309 │
│ Baseline      ┆ 100                       ┆ 12301          ┆ 0.831363 ┆ 193.497407  │
└───────────────┴───────────────────────────┴────────────────┴──────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x2861ca6d0>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='center', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Hidden_Layer_Sizes <br> Parameter</center>', type='html'), column_align='center', column_width=None), ColInfo(var='Num_Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Total Number of<br>Internal Parameters</center>', type='html'), column_align='center', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='center', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Layers With Constant Neurons', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x2861cae50>, _formats=[<great_tables._gt_data.FormatInfo object at 0x2861cac90>, <great_tables._gt_data.FormatInfo object at 0x2861caf10>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='tab

## Layers With Constant Number of Parameters

In [11]:
(
    result_table['test_results_parmeters-hidden_layer_sizes']
    .tab_header(title="Layers With Constant Number of Parameters", subtitle="Test Set Prediction Results")
    .cols_label(
        Num_Parameters = html("<center>Total Number of<br>Internal Parameters</center>")
    )
    # .save(file="./tables_and_figures/test_results_parmeters-hidden_layer_sizes.png")
)

GT(_tbl_data=shape: (2, 7)
┌───────────────┬──────────────────┬────────────────┬──────────┬──────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters       ┆ Num_Parameters ┆ Recall   ┆ ROC_AUC  ┆ Accuracy ┆ Fit_Time   │
│ ---           ┆ ---              ┆ ---            ┆ ---      ┆ ---      ┆ ---      ┆ ---        │
│ str           ┆ str              ┆ i64            ┆ f64      ┆ f64      ┆ f64      ┆ f64        │
╞═══════════════╪══════════════════╪════════════════╪══════════╪══════════╪══════════╪════════════╡
│               ┆ [47, 46, 46, 46] ┆ 12313          ┆ 0.718652 ┆ 0.782535 ┆ 0.729182 ┆ 878.951744 │
│ Baseline      ┆ 100              ┆ 12301          ┆ 0.713213 ┆ 0.808297 ┆ 0.745176 ┆ 193.497407 │
└───────────────┴──────────────────┴────────────────┴──────────┴──────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x2861cb810>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='left', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Hidden_Layer_Sizes <br> Parameter', type='html'), column_align='left', column_width=None), ColInfo(var='Num_Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Total Number of<br>Internal Parameters</center>', type='html'), column_align='right', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='right', column_width=None), ColInfo(var='ROC_AUC', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='ROC AUC', type='html'), column_align='right', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'ROC_AUC', 'Accuracy', 'Fit_Time'], built=None)]), _heading=Heading(title='Layers With Constant Number of Parameters', subtitle='Test Set Prediction Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x2861cbe90>, _formats=[<great_tables._gt_data.FormatInfo object at 0x286200890>, <great_tables._gt_data.FormatInfo object at 0x286200d10>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333

In [12]:
(
    result_table['grid_results_parmeters-hidden_layer_sizes']
    .tab_header(title="Layers With Constant Number of Parameters", subtitle="Cross Validation Results")
    .cols_label(
        Num_Parameters = html("<center>Total Number of<br>Internal Parameters</center>")
    )
    # .save(file="./tables_and_figures/grid_results_parmeters-hidden_layer_sizes.png")
)

GT(_tbl_data=shape: (5, 5)
┌───────────────┬──────────────────────┬────────────────┬──────────┬────────────┐
│ Grid_Variable ┆ Parameters           ┆ Num_Parameters ┆ Recall   ┆ Fit_Time   │
│ ---           ┆ ---                  ┆ ---            ┆ ---      ┆ ---        │
│ str           ┆ str                  ┆ i64            ┆ f64      ┆ f64        │
╞═══════════════╪══════════════════════╪════════════════╪══════════╪════════════╡
│               ┆ [47, 46, 46, 46]     ┆ 12313          ┆ 0.927616 ┆ 878.951744 │
│               ┆ [42, 42, 42, 41, 42] ┆ 12306          ┆ 0.926233 ┆ 709.725877 │
│               ┆ [53, 53, 54]         ┆ 12299          ┆ 0.908242 ┆ 752.372832 │
│               ┆ [65, 65]             ┆ 12286          ┆ 0.884538 ┆ 612.875343 │
│ Baseline      ┆ 100                  ┆ 12301          ┆ 0.831363 ┆ 193.497407 │
└───────────────┴──────────────────────┴────────────────┴──────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x286201110>, _boxhead=Boxhead([ColInfo(var='Grid_Variable', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='', type='html'), column_align='center', column_width=None), ColInfo(var='Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Hidden_Layer_Sizes <br> Parameter</center>', type='html'), column_align='center', column_width=None), ColInfo(var='Num_Parameters', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='<center>Total Number of<br>Internal Parameters</center>', type='html'), column_align='center', column_width=None), ColInfo(var='Recall', type=<ColInfoTypeEnum.default: 1>, column_label='Recall', column_align='center', column_width=None), ColInfo(var='Fit_Time', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Fit Time (s)', type='html'), column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([SpannerInfo(spanner_id='Metrics', spanner_level=0, spanner_label='Metrics', spanner_units=None, spanner_pattern=None, vars=['Recall', 'Fit_Time'], built=None)]), _heading=Heading(title='Layers With Constant Number of Parameters', subtitle='Cross Validation Results', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x2862016d0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x2862013d0>, <great_tables._gt_data.FormatInfo object at 0x286201910>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=